PostgreSQL’s documentation does an excellent job of introducing the concept of Window Functions: 

*** a window function performs a calculation across a set of table rows that are somehow related to the current row. ***

 This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.

Through introducing [window functions](https://www.postgresql.org/docs/9.1/static/tutorial-window.html), we have also introduced two statements that you may not be familiar with: OVER and PARTITION BY. These are key to window functions. Not every window function uses PARTITION BY; we can also use ORDER BY or no statement at all depending on the query we want to run. You will practice using these clauses in the upcoming quizzes. If you want more details right now, [this resource](https://blog.sqlauthority.com/2015/11/04/sql-server-what-is-the-over-clause-notes-from-the-field-101/) from Pinal Dave is helpful.

Note: You can’t use window functions and standard aggregations in the same query. More specifically, you can’t include window functions in a GROUP BY clause.

The OVER clause has three components: partitioning, ordering, and framing. Partitioning is always supported, but support for ordering and framing depends on which type of window function you are using.

The partitioning component, expressed as PARTITION BY, is optional and is supported for all types of window functions. Partitioning divides up the rows. It’s kind of like a physical window that is divided up into panes. The entire window is a window. The panes are also windows. Say that I partition by CustomerID. The partition for CustomerID 1000 is restricted to rows for CustomerID 1000.  For the calculation on any given row, the window consists of rows from that partition only. This means that the ROW_NUMBER function restarts the row numbers for each partition, for example. You can specify more than one column for PARTITION BY as long as you separate the columns with commas. Partitioning is not the same as grouping with the GROUP BY clause. When you use the GROUP BY clause, only one row is returned for each group. Partitioning is just a way to divide the data in order to perform the calculations; the detail columns are returned. 
![Figure 1 shows an example](https://blog.sqlauthority.com/i/a/101-1.png).

Ordering within the OVER clause is supported when the data must be sorted to perform the calculation. For example, ROW_NUMBER requires an ORDER BY expression within the OVER clause because the rows must be lined up. Actually, most of the window functions support ORDER BY.  ORDER BY in the OVER clause is not supported for calculating subtotals, for example. You don’t need the data sorted to calculate a sum. Note that the ORDER BY within the OVER clause has nothing to do with an ORDER BY clause found in the query itself. Just like partitioning, you can specify a comma delimited list of columns. Figure 2 shows the results of ORDER BY in the OVER clause.
![Fig.2](https://blog.sqlauthority.com/i/a/101-2.png)

The third component of the OVER CLAUSE is called framing. Framing was introduced with SQL Server 2012 and is used in only a couple of situations: accumulating window aggregates (running totals, moving averages, etc.) and the functions FIRST_VALUE and LAST_VALUE. While partitioning is similar to window panes, framing is like a stained glass window. When performing the calculations, every row might see a different set of rows through the window. For example, when calculating a running total, row 1 sees row 1. Row 2 sees rows 1 and 2. Row 3 see rows 1, 2, and 3. The syntax for framing is a bit complex, so I am going to save it for my next post. By default, the frame consists of the first row of the sorted partition and all subsequent rows up to the current row.  Figure 3 is an example. The small arrows represent the current row where the calculation is being performed.
![Fig.3](https://blog.sqlauthority.com/i/a/101-3.png)

In [ ]:
SELECT CustomerID, SalesOrderID, TotalDue,
ROW_NUMBER() OVER(PARTITION BY CustomerID ORDER BY SalesOrderID) AS RowNum,
SUM(TotalDue) OVER() AS GrandTotal,
SUM(TotalDue) OVER(PARTITION BY CustomerID) AS SubTotal,
SUM(TotalDue) OVER(PARTITION BY CustomerID ORDER BY SalesOrderID) AS RunningTotal,
LAG(SalesOrderID) OVER(PARTITION BY CustomerID ORDER BY SalesOrderID) AS PrevOrder,
FIRST_VALUE(TotalDue) OVER(PARTITION BY CustomerID ORDER BY SalesOrderID) AS FirstAmt
FROM Sales.SalesOrderHeader;

Creating a Running Total Using Window Functions:
Using Derek's previous video as an example, create another running total. This time, create a running total of standard_amt_usd (in the orders table) over order time with no date truncation. Your final table should have two columns: one with the amount being added for each new row, and a second with the running total.

In [ ]:
select standard_amt_usd, occurred_at,
sum(standard_amt_usd) over(order by occurred_at) running_total
from orders

Create a running total of `standard_qty` from `orders` table over resetting at starting of the month

In [ ]:
select standard_qty, occurred_at,
    date_trunc('month', occurred_at) ordered_month,
    sum(standard_qty) 
        over(partition by date_trunc('month', occurred_at)
            order by occurred_at) running_total
from orders

In [ ]:
## row_number(), rank(), dense_rank()

select id, account_id, date_trunc('month', occurred_at),
    row_number() over(partition by account_id 
        order by date_trunc('month', occurred_at)) --'row number reset over account id',
    rank()  over(partition by account_id 
        order by date_trunc('month', occurred_at)) --'rank repeated for same order and skipped says how many entities before it or the position'
     rank()  over(partition by account_id 
        order by date_trunc('month', occurred_at)) --'rank repeated for same order but not skipped or actual rank'   


from orders

## Quiz: ROW_NUMBER & RANK
Ranking Total Paper Ordered by Account
Select the id, account_id, and total variable from the orders table, then create a column called total_rank that ranks this total amount of paper ordered (from highest to lowest) for each account using a partition. Your final table should have these four columns.

In [ ]:
select id, account_id, total,
	rank() over(partition by account_id 
               order by total desc) total_rank
from orders

# Aggregates in Window Functions with and without ORDER BY



In [ ]:
SELECT id,
       account_id,
       standard_qty,
       DATE_TRUNC('month', occurred_at) AS month,
       DENSE_RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS dense_rank,
       SUM(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS sum_std_qty,
       COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS count_std_qty,
       AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS avg_std_qty,
       MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS min_std_qty,
       MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS max_std_qty
FROM orders

Aggregates in Window Functions with and without ORDER BY
The ORDER BY clause is one of two clauses integral to window functions. The ORDER and PARTITION define what is referred to as the “window”—the ordered subset of data over which calculations are made. `Removing ORDER BY just leaves an unordered partition`; in our query's case, each column's value is simply an aggregation (e.g., sum, count, average, minimum, or maximum) of all the standard_qty values in its respective account_id.

As Stack Overflow user mathguy explains:
```
The easiest way to think about this - leaving the ORDER BY out is equivalent to "ordering" in a way that all rows in the partition are "equal" to each other. Indeed, you can get the same effect by explicitly adding the ORDER BY clause like this: ORDER BY 0 (or "order by" any constant expression), or even, more emphatically, ORDER BY NULL.

Why you get the COUNT() or SUM() etc. for the entire partition has to do with the default windowing clause: RANGE between unbounded preceding and current row. "Range" (as opposed to "ROWS") means all rows "tied" with the current row are also included, even if they don't precede it. Since all rows are tied, this means the entire partition is included, no matter which row is "current."
```

The easiest way to think about this - leaving the ORDER BY out is equivalent to "ordering" in a way that all rows in the partition are "equal" to each other. Indeed, you can get the same effect by explicitly adding the ORDER BY clause like this: ORDER BY 0 (or "order by" any constant expression), or even, more emphatically, ORDER BY NULL.

Shorten Your Window Function Queries by Aliasing
QUIZ QUESTION
Using Derek's example in the previous window, deconstruct the window function alias into its two parts: the alias part and the window function part.



PART

QUERY TEXT

Alias part

`window main_window as`

Window function part

```
(PARTITION BY account_id ORDER BY DATE_TRUNC('year',occurred_at))
```

usage

`select function() over main_window as name`

Now, create and use an alias to shorten the following query (which is different than the one in Derek's previous video) that has multiple window functions. Name the alias account_year_window, which is more descriptive than main_window in the example above.

In [ ]:
SELECT id,
       account_id,
       DATE_TRUNC('year',occurred_at) AS year,
       DENSE_RANK() over main_window dense_rank,
       total_amt_usd,
       SUM(total_amt_usd) over main_window AS sum_total_amt_usd,
       COUNT(total_amt_usd) over main_window AS count_total_amt_usd,
       AVG(total_amt_usd) over main_window AS avg_total_amt_usd,
       MIN(total_amt_usd) over main_window AS min_total_amt_usd,
       MAX(total_amt_usd) over main_window AS max_total_amt_usd
FROM orders
window main_window as (PARTITION BY account_id ORDER BY DATE_TRUNC('year',occurred_at))